In [ ]:
from crystalbuilder import *
import crystalbuilder.geometry as geo
import matplotlib.pyplot as plt
import tidy3d as td
import crystalbuilder.convert as cv
import numpy as np
import vedo

Here we will create a simulation of the core-shell diamond crystal. This is a rod-connected diamond with a cubic conventional cell. We'll set it up in the usual manner, but we can use an actual distance in nanometers for our lattice constant. `a_mag` is this constant in microns (Tidy3D default units)

In [ ]:
lattice_constant = 500 #nm

a1 = [1, 0, 0]
a2 = [0, 1 ,0]
a3 = [0, 0, 1]

a_mag = lattice_constant/1000 #convert to microns
geo_lattice = lattice.Lattice(a1, a2, a3, magnitude = [a_mag, a_mag, a_mag])

We'll load our points from Bilbao (or a local copy if already done before). We don't scale any of this using the lattice constant.  

In [ ]:
diamond = bilbao.SpaceGroup(227)
pt = 1/8
points = diamond.calculate_points(point_list=[(pt, pt, pt)]) #This is the 8a wyckoff position

The IUCr tabulated general positions are in units of the lattice constant. After we connect the nearest neighbors, we convert to our micron units and scale the structure. This is done by passing `a_mag` to `NearestNeighbors()`. 

In [ ]:
radius = .04 #microns

crystal_core = geo.NearestNeighbors(points, radius=radius, neighborhood_range=.45, a_mag=a_mag)

For our core-shell structure, we will just create a slightly larger (by `shell_radius`) set of connected rods. 

In [ ]:
shell_radius = .02 #microns

crystal_shell = geo.NearestNeighbors(points, radius=(radius+shell_radius), neighborhood_range=.45, a_mag=a_mag)

The geometry is just a combination of these two lattices, **making sure to include the core *after* the shell**. In Tidy3D, items later in the list will override earlier ones.

In [ ]:
geometry = [crystal_shell, crystal_core]